In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

The following dictionary is from https://gist.github.com/rogerallen/1583593

In [4]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
us_state_unabbrev = dict([(j,i) for i,j in us_state_abbrev.items()])

In [5]:
df_amtrak = pd.read_csv('Amtrak_Stations.csv')
df_amtrak = df_amtrak[['STATE','STNNAME']].set_index('STATE')
df_stations = pd.DataFrame(df_amtrak.groupby(['STATE']).count())
df_stations.reset_index(inplace=True)
df_stations.rename(columns={'STATE':'state','STNNAME':'num_stns'},inplace=True)
df_stations.replace({'state': us_state_unabbrev}, inplace=True)
df_stations.set_index('state',inplace=True)
df_stations.head()

,num_stns
state,
Alabama,5
Arkansas,6
Arizona,8
California,76
Colorado,10


In [7]:
age = pd.read_csv('A102102 - Age of worker (8) (Workers 16 years and over).csv',skiprows=2)
age = age[age.Output != 'Margin of Error']
age.drop(['Output'], axis=1, inplace=True)
age.rename(columns={'RESIDENCE':'state','Age of Worker 8':'age',\
                    'Workers 16 and Over':'num_workers_by_age'},inplace=True)
age['num_workers_by_age'] = pd.to_numeric(age['num_workers_by_age'], errors='coerce')
age.set_index('state',inplace=True)
age.head()

,age,num_workers_by_age
state,,
Alabama,"Total, all ages 16 years and over",2014965.0
Alabama,16 and 17 years,16950.0
Alabama,18 to 24 years,247090.0
Alabama,25 to 44 years,876485.0
Alabama,45 to 59 years,642270.0


In [8]:
total_age = age.num_workers_by_age[age['age'] == 'Total, all ages 16 years and over']
df_age=pd.DataFrame(total_age)
df_age.rename(columns={'num_workers_by_age':'total_workers_by_age'},inplace=True)
df_age['age_16_17'] = age.num_workers_by_age[age['age'] == '16 and 17 years']/df_age['total_workers_by_age']
df_age['age_18_24'] = age.num_workers_by_age[age['age'] == '18 to 24 years']/df_age['total_workers_by_age']
df_age['age_25_44'] = age.num_workers_by_age[age['age'] == '25 to 44 years']/df_age['total_workers_by_age']
df_age['age_45_59'] = age.num_workers_by_age[age['age'] == '45 to 59 years']/df_age['total_workers_by_age']
df_age['age_60_64'] = age.num_workers_by_age[age['age'] == '60 to 64 years']/df_age['total_workers_by_age']
df_age['age_65_74'] = age.num_workers_by_age[age['age'] == '65 to 74 years']/df_age['total_workers_by_age']
df_age['age_75'] = age.num_workers_by_age[age['age'] == '75 years and over']/df_age['total_workers_by_age']
df_age.head()

,total_workers_by_age,age_16_17,age_18_24,age_25_44,age_45_59,age_60_64,age_65_74,age_75
state,,,,,,,,
Alabama,2014965.0,0.008412,0.122627,0.434988,0.318750,0.063862,0.043088,0.008271
Alaska,359985.0,0.011820,0.140950,0.437310,0.302777,0.064489,0.038349,0.004292
Arizona,2843105.0,0.009794,0.134680,0.441099,0.301908,0.063432,0.041697,0.007390
Arkansas,1248000.0,0.010978,0.125573,0.434467,0.310733,0.063946,0.045304,0.008998
California,17193695.0,0.004749,0.120122,0.458272,0.309654,0.061012,0.039218,0.006974


In [9]:
workers = pd.read_csv('A302100 - Total Workers (1) (Workers 16 years and over).csv',skiprows=2)
workers = workers[workers.Output != 'Margin of Error']
workers.drop(['Output'], axis=1, inplace=True)
workers.rename(columns={'RESIDENCE':'state','Workers 16 and Over':'num_workers'\
                        ,'WORKPLACE': 'work_place'},inplace=True)
workers.fillna(0)
workers['away'] = (workers['state'] != workers['work_place'])
workers['num_away'] = workers['away']*workers['num_workers']
workers.drop(workers.tail(1).index,inplace=True)
workers.fillna(0,inplace=True)
workers = workers[workers.state !=0]
workers.head(10)

,state,work_place,num_workers,away,num_away
0,Alabama,Alabama,1921785.0,False,0.0
2,Alabama,Alaska,80.0,True,80.0
4,Alabama,Arizona,160.0,True,160.0
6,Alabama,Arkansas,290.0,True,290.0
8,Alabama,California,535.0,True,535.0
10,Alabama,Colorado,215.0,True,215.0
12,Alabama,Connecticut,20.0,True,20.0
14,Alabama,Delaware,0.0,True,0.0
16,Alabama,District of Columbia,320.0,True,320.0
18,Alabama,Florida,9100.0,True,9100.0


In [10]:
a_workers = pd.DataFrame(workers.groupby(['state'])['num_workers'].sum())
a_workers.rename(columns={'num_workers':'tot_num_workers'},inplace=True)
a_workers.head()

,tot_num_workers
state,
Alabama,2014190.0
Alaska,359777.0
Arizona,2840205.0
Arkansas,1247663.0
California,17180945.0


In [11]:
workers_new = workers.reset_index()
workers_new.head()
away_workers = pd.DataFrame(workers_new.groupby(['state'])['num_away'].sum())
away_workers.head()

,num_away
state,
Alabama,92405.0
Alaska,1272.0
Arizona,45020.0
Arkansas,46213.0
California,68480.0


In [12]:
vehicles = pd.read_csv('EA_A113201 - Vehicles available (6) by Poverty status (4) (Households for which poverty status is determined).csv'\
                       , skiprows =2)
vehicles = vehicles[vehicles.Output != 'Margin of Error']
vehicles.drop(['Output'], axis=1, inplace=True)
vehicles.rename(columns={'RESIDENCE':'state','Poverty Status 4':'pov_status'\
                        ,'Households with Poverty Status': 'households'\
                        ,'Vehicles Available 6':'num_vehicles'},inplace=True)
vehicles.fillna(0)
vehicles.set_index('state',inplace=True)
vehicles.head(13)

,num_vehicles,pov_status,households
state,,,
Alabama,"Total, vehicles available","Total, poverty status",1851060.0
Alabama,"Total, vehicles available",Below 100 percent of the poverty level,331975.0
Alabama,"Total, vehicles available",100 to 149 percent of the poverty level,205790.0
Alabama,"Total, vehicles available",At or above 150 percent of the poverty level,1313300.0
Alabama,0 vehicles,"Total, poverty status",119375.0
Alabama,0 vehicles,Below 100 percent of the poverty level,63295.0
Alabama,0 vehicles,100 to 149 percent of the poverty level,22075.0
Alabama,0 vehicles,At or above 150 percent of the poverty level,34005.0
Alabama,1 vehicle,"Total, poverty status",609740.0


In [13]:
total_hholds = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='Total, vehicles available')]
df_vehicles = pd.DataFrame(total_hholds)
df_vehicles.rename(columns={'households':'total_hholds'}, inplace=True)
df_vehicles['below_100'] = vehicles.households[(vehicles['pov_status']=='Below 100 percent of the poverty level') \
                            & (vehicles['num_vehicles']=='Total, vehicles available')]
df_vehicles['below_150'] = vehicles.households[(vehicles['pov_status']=='100 to 149 percent of the poverty level') \
                            & (vehicles['num_vehicles']=='Total, vehicles available')]
df_vehicles['below_150_perc_pov'] = (df_vehicles['below_150']+df_vehicles['below_100'])/df_vehicles['total_hholds']
df_vehicles.drop(['below_150','below_100'],axis=1,inplace=True)
df_vehicles.head()

,total_hholds,below_150_perc_pov
state,,
Alabama,1851060.0,0.290517
Alaska,250235.0,0.158731
Arizona,2448920.0,0.251446
Arkansas,1141480.0,0.301433
California,12807385.0,0.232380


In [14]:
df_vehicles['0_vehicles'] = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='0 vehicles')]/df_vehicles['total_hholds']
df_vehicles['1_vehicles'] = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='1 vehicle')]/df_vehicles['total_hholds']
df_vehicles['2_vehicles'] = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='2 vehicles')]/df_vehicles['total_hholds']
df_vehicles['3_vehicles'] = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='3 vehicles')]/df_vehicles['total_hholds']
df_vehicles['more_vehicles'] = vehicles.households[(vehicles['pov_status']=='Total, poverty status') \
                            & (vehicles['num_vehicles']=='4-or-more vehicles')]/df_vehicles['total_hholds']
df_vehicles.head()

,total_hholds,below_150_perc_pov,0_vehicles,1_vehicles,2_vehicles,3_vehicles,more_vehicles
state,,,,,,,
Alabama,1851060.0,0.290517,0.064490,0.329400,0.369694,0.160867,0.075551
Alaska,250235.0,0.158731,0.094771,0.308370,0.376326,0.149320,0.071213
Arizona,2448920.0,0.251446,0.067354,0.372364,0.381748,0.127683,0.050851
Arkansas,1141480.0,0.301433,0.063935,0.340164,0.388128,0.149148,0.058621
California,12807385.0,0.232380,0.076089,0.317117,0.374471,0.153592,0.078730
Colorado,2051615.0,0.192629,0.054355,0.305915,0.401791,0.164307,0.073632
Connecticut,1354715.0,0.168408,0.090521,0.325578,0.378733,0.143573,0.061592
Delaware,348050.0,0.187574,0.063884,0.339779,0.397156,0.140511,0.058670
District of Columbia,276545.0,0.226202,0.363973,0.437198,0.160679,0.028748,0.009420


In [15]:
minority = pd.read_csv('EA_B302105 - Minority status (3) (Workers 16 years and over).csv',skiprows=2)
minority = minority[minority.Output != 'Margin of Error']
minority.drop(['Output'], axis=1, inplace=True)
minority.rename(columns={'RESIDENCE':'state','WORKPLACE':'work_place'\
                        ,'Minority Status 3': 'minority_status'\
                        ,'Workers 16 and Over':'num_workers'},inplace=True)
minority.fillna(0)
minority.set_index('state',inplace=True)
minority.head()

,work_place,minority_status,num_workers
state,,,
Alabama,Alabama,Total persons,1921670.0
Alabama,Alabama,"White alone, not Hispanic/Latino",1322775.0
Alabama,Alabama,Other,598900.0
Alabama,Alaska,Total persons,80.0
Alabama,Alaska,"White alone, not Hispanic/Latino",65.0


In [16]:
work_pkace = minority.work_place[minority.minority_status=='Other']
df_minority = pd.DataFrame(work_pkace)
df_minority['minorities'] = minority.num_workers[minority.minority_status=='Other']
df_minority['tot'] = minority.num_workers[minority.minority_status=='Total persons']
df_minority.reset_index(inplace=True)
df_minority.head()

,state,work_place,minorities,tot
0,Alabama,Alabama,598900.0,1921670.0
1,Alabama,Alaska,20.0,80.0
2,Alabama,Arizona,60.0,160.0
3,Alabama,Arkansas,30.0,290.0
4,Alabama,California,105.0,535.0


In [17]:
a_minority = pd.DataFrame(df_minority.groupby(['state'])['tot'].sum())
a_minority.columns =['tot_persons_by_minor']
a_minority['minorities'] = df_minority.groupby(['state'])['minorities'].sum()
a_minority.head()

,tot_persons_by_minor,minorities
state,,
Alabama,2014055.0,624356.0
Alaska,359772.0,113452.0
Arizona,2839965.0,1167049.0
Arkansas,1247603.0,301087.0
California,17179445.0,10123039.0


In [19]:
poverty = pd.read_csv('EA_B304100 - Poverty status (4) (Workers 16 years and over for whom poverty status is determined).csv',skiprows=2)
poverty = poverty[poverty.Output != 'Margin of Error']
poverty.drop(['Output'], axis=1, inplace=True)
poverty.rename(columns={'RESIDENCE':'state','WORKPLACE':'work_place'\
                        ,'Poverty Status 4': 'poverty_status'\
                        ,'Workers with Poverty Status':'num_workers'},inplace=True)
poverty.fillna(0)
poverty.set_index('state',inplace=True)
poverty.head()

,work_place,poverty_status,num_workers
state,,,
Alabama,Alabama,"Total, poverty status",1910870.0
Alabama,Alabama,Below 100 percent of the poverty level,160940.0
Alabama,Alabama,100 to 149 percent of the poverty level,150235.0
Alabama,Alabama,At or above 150 percent of the poverty level,1599695.0
Alabama,Alaska,"Total, poverty status",80.0


In [20]:
poverty.reset_index(inplace=True)
poverty.set_index(['state','work_place'],inplace=True)
below100 = poverty.num_workers[poverty.poverty_status=='Below 100 percent of the poverty level']
df_poverty = pd.DataFrame(below100)
df_poverty.rename(columns={'num_workers':'below_100'},inplace=True)
df_poverty['below_150']=poverty.num_workers[poverty.poverty_status=='100 to 149 percent of the poverty level']
df_poverty['tot_below_150']=df_poverty['below_100']+df_poverty['below_150']
df_poverty['total_poverty']=poverty.num_workers[poverty.poverty_status=='Total, poverty status']
df_poverty=df_poverty[['total_poverty','tot_below_150']]
df_poverty.head()

total_poverty  tot_below_150
state   work_place                              
Alabama Alabama         1910870.0       311175.0
        Alaska               80.0            0.0
        Arizona             160.0           10.0
        Arkansas            290.0           30.0
        California          535.0           40.0

In [21]:
transport = pd.read_csv('NHTS_A102106 - Means of transportation (13) (Workers 16 years and over).csv',skiprows=2)
transport = transport[transport.Output=='Estimate']
transport.drop(['Output'],axis=1,inplace=True)
transport.rename(columns={'RESIDENCE':'state','Means of Transportation 18':\
                         'means_transp','Workers 16 and Over':'num_workers'}\
                 ,inplace=True)
transport.set_index('state', inplace=True)
transport['num_workers'] = pd.to_numeric(transport['num_workers'], errors='coerce')
transport.head()

,means_transp,num_workers
state,,
Alabama,Railroad,70.0
Alaska,Railroad,4.0
Arizona,Railroad,1030.0
Arkansas,Railroad,60.0
California,Railroad,71910.0


In [23]:
list_transp_means = transport.means_transp.unique()
total_transp = transport.num_workers[transport.means_transp=='Total']
df_transp = pd.DataFrame(total_transp)
df_transp = df_transp.rename(columns={'num_workers':'total_transp'})
for i in list_transp_means[:-1]:
    df_transp[i]=transport.num_workers[transport.means_transp==i]/df_transp['total_transp']
df_transp.head()

,total_transp,Railroad,Ferryboat,Bicycle,Walked,Taxicab,Motorcycle,Other method,Worked at home,Carpool,Drove Alone,Subway or Streetcar or Lightrail,Bus
state,,,,,,,,,,,,,
Alabama,2014965.0,0.000035,0.000134,0.001146,0.011420,0.000392,0.001392,0.007578,0.029276,0.087622,0.857280,0.000094,0.003628
Alaska,359985.0,0.000011,0.000639,0.009667,0.078628,0.002861,0.001833,0.039668,0.046044,0.124908,0.681334,0.000375,0.014042
Arizona,2843105.0,0.000362,0.000047,0.009611,0.019924,0.000825,0.004754,0.012150,0.057010,0.108712,0.766658,0.001001,0.018946
Arkansas,1248000.0,0.000048,0.000036,0.001534,0.017500,0.000601,0.002200,0.007973,0.031631,0.107564,0.826895,0.000120,0.003886
California,17193695.0,0.004182,0.000546,0.011058,0.026950,0.000693,0.003526,0.010266,0.054348,0.106173,0.734944,0.012130,0.035184


In [28]:
worker_sex = pd.read_csv('A202111 - Sex (3) (Workers 16 years and over).csv',skiprows=2)
worker_sex = worker_sex[worker_sex.Output=='Estimate'].set_index('WORKPLACE')
df_sex = pd.DataFrame(worker_sex['Workers 16 and Over'][worker_sex['Sex 3']=='Female'])
df_sex['male'] = worker_sex['Workers 16 and Over'][worker_sex['Sex 3']=='Male']
df_sex['both_sexes'] = worker_sex['Workers 16 and Over'][worker_sex['Sex 3']=='Both sexes']
df_sex.rename(columns={'Workers 16 and Over':'female'},inplace=True)
df_sex['female'] = df_sex['female']/df_sex['both_sexes']
df_sex['male'] = df_sex['male']/df_sex['both_sexes']
df_sex.drop(['both_sexes'],axis=1,inplace=True)
df_sex.head()

,female,male
WORKPLACE,,
Alabama,0.473344,0.526658
Alaska,0.437402,0.562585
Arizona,0.463585,0.536415
Arkansas,0.468111,0.531889
California,0.453283,0.546717


In [29]:
table_1 = pd.concat([a_workers,away_workers,df_sex,a_minority,df_age, df_vehicles, df_transp, df_stations], axis=1, join='outer')
table_1.fillna(0,inplace=True)
table_1['num_away']=table_1['num_away']/table_1['tot_num_workers']
table_1=table_1.drop(['total_workers_by_age','tot_num_workers','total_hholds','total_transp',\
                      'tot_persons_by_minor'],axis=1)
table_1.head()

,num_away,female,male,minorities,age_16_17,age_18_24,age_25_44,age_45_59,age_60_64,age_65_74,...,Walked,Taxicab,Motorcycle,Other method,Worked at home,Carpool,Drove Alone,Subway or Streetcar or Lightrail,Bus,num_stns
Alabama,0.045877,0.473344,0.526658,624356.0,0.008412,0.122627,0.434988,0.318750,0.063862,0.043088,...,0.011420,0.000392,0.001392,0.007578,0.029276,0.087622,0.857280,0.000094,0.003628,5.0
Alaska,0.003536,0.437402,0.562585,113452.0,0.011820,0.140950,0.437310,0.302777,0.064489,0.038349,...,0.078628,0.002861,0.001833,0.039668,0.046044,0.124908,0.681334,0.000375,0.014042,0.0
Arizona,0.015851,0.463585,0.536415,1167049.0,0.009794,0.134680,0.441099,0.301908,0.063432,0.041697,...,0.019924,0.000825,0.004754,0.012150,0.057010,0.108712,0.766658,0.001001,0.018946,8.0
Arkansas,0.037040,0.468111,0.531889,301087.0,0.010978,0.125573,0.434467,0.310733,0.063946,0.045304,...,0.017500,0.000601,0.002200,0.007973,0.031631,0.107564,0.826895,0.000120,0.003886,6.0
California,0.003986,0.453283,0.546717,10123039.0,0.004749,0.120122,0.458272,0.309654,0.061012,0.039218,...,0.026950,0.000693,0.003526,0.010266,0.054348,0.106173,0.734944,0.012130,0.035184,76.0


In [30]:
table_1.to_csv('table_1.csv')